# ML Pipeline example - XGBoost Training

In [1]:
# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio 

### Install and register package dependencied and build commands
Those will convert to container build instructions 

In [2]:
%%nuclio cmd 
pip install sklearn
pip install xgboost
pip install matplotlib

In [3]:
%nuclio config spec.build.baseImage = "python:3.6-jessie"
#%nuclio config spec.image = ".mlrun/xgb:latest"

%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


## ML Training code

In [4]:
import xgboost as xgb
import os
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score

dtrain = dtest = Y_test = None

def load_dataset():
    global dtrain, dtest, Y_test
    iris = load_iris()
    y = iris['target']
    X = iris['data']
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)
    dtrain = xgb.DMatrix(X_train, label=Y_train)
    dtest = xgb.DMatrix(X_test, label=Y_test)


def xgb_train(context, model_name='model.bst',
            max_depth=6,
            num_class=10,
            eta=0.2,
            gamma=0.1,
            steps=20):
    global dtrain, dtest, Y_test

    if dtrain is None:
        load_dataset()

    # Get params from event
    param = {"max_depth": max_depth,
             "eta": eta, "nthread": 4,
             "num_class": num_class,
             "gamma": gamma,
             "objective": "multi:softprob"}

    # Train model
    xgb_model = xgb.train(param, dtrain, steps)

    preds = xgb_model.predict(dtest)
    best_preds = np.asarray([np.argmax(line) for line in preds])

    context.log_result('accuracy', float(accuracy_score(Y_test, best_preds)))

    os.makedirs('models', exist_ok=True)
    model_file = model_name #os.path.join('models', model_name)
    xgb_model.save_model(model_file)
    context.log_artifact('model', src_path=model_file, labels={'framework': 'xgboost'})

from mlrun.artifacts import PlotArtifact
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from io import BytesIO

def plot_iter(context, iterations, col='accuracy', num_bins=10):
    df = pd.read_csv(BytesIO(iterations.get()))
    x = df['output.{}'.format(col)]
    fig, ax = plt.subplots(figsize=(6,6))
    n, bins, patches = ax.hist(x, num_bins, density=1)
    ax.set_xlabel('Accuraccy')
    ax.set_ylabel('Count')
    context.log_artifact(PlotArtifact('myfig', body=fig))

In [5]:
# nuclio: end-code
# (end-code marker tells nuclio to stop parsing the notebook from this cell)

In [6]:
from mlrun import new_function, code_to_function, NewTask, get_run_db, mlconf, mount_v3io, new_model_server
mlconf.dbpath = '/User/mlrun'
#mlconf.package_path = 'git+https://github.com/mlrun/mlrun.git@development'
import kfp
from kfp import dsl

## Test the code locally 

In [7]:
task = NewTask(handler=xgb_train, out_path='/User/mlrun/data').with_hyper_params({'eta': [0.1, 0.2, 0.3]}, selector='max.accuracy')
run = new_function().run(task)

[mlrun] 2019-09-24 03:37:59,018 starting run None uid=652cd348e5224c3091a4c30bb2740b38


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...740b38,0,Sep 24 03:37:59,completed,,kind=handlerowner=iguazio,,,best_iteration=1accuracy=0.9666666666666667,modeliteration_results


type result.show() to see detailed results/progress or use CLI:
!mlrun get run --uid 652cd348e5224c3091a4c30bb2740b38 
[mlrun] 2019-09-24 03:37:59,280 run executed, status=completed


## Create a containerized function from the notebook code
We create a function object which defined the code, metadata, execution and build instructions <br>

later on we build the image (so we dont have to repeat this every run)

In [8]:
# create a job from the notebook, attache it to iguazio data fabric (v3io)
fn = code_to_function('training')

In [9]:
fn.build(image='mlrun/xgb:latest')

[mlrun] 2019-09-24 03:38:03,664 building image (mlrun/xgb:latest)
FROM python:3.6-jessie
WORKDIR /run
RUN pip install sklearn
RUN pip install xgboost
RUN pip install matplotlib
RUN pip install git+https://github.com/mlrun/mlrun.git@development
ENV PYTHONPATH /run
[mlrun] 2019-09-24 03:38:03,666 using in-cluster config.
[mlrun] 2019-09-24 03:38:03,685 Pod mlrun-build-trt6b created
..
INFO[0000] Downloading base image python:3.6-jessie     
2019/09/24 03:38:07 No matching credentials were found, falling back on anonymous
INFO[0000] Unpacking rootfs as cmd RUN pip install sklearn requires it. 
INFO[0010] Taking snapshot of full filesystem...        
INFO[0020] Skipping paths under /kaniko, as it is a whitelisted directory 
INFO[0020] Skipping paths under /empty, as it is a whitelisted directory 
INFO[0020] Skipping paths under /var/run, as it is a whitelisted directory 
INFO[0020] Skipping paths under /dev, as it is a whitelisted directory 
INFO[0020] Skipping paths under /proc, as it is 

## Create and run the pipeline

In [30]:
artifacts_path = 'v3io:///users/admin/mlrun/kfp/{{workflow.uid}}/'

In [31]:
@dsl.pipeline(
    name='My XGBoost training pipeline',
    description='Shows how to use mlrun.'
)
def xgb_pipeline(
   eta = [0.1, 0.2, 0.3], gamma = [0.0, 0.1, 0.2, 0.3]
):
    fn.with_code()  # update the code from notebook
    train = fn.to_step(
        NewTask(handler='xgb_train', out_path=artifacts_path, outputs=['model'])\
                .with_hyper_params({'eta': eta, 'gamma': gamma}, selector='max.accuracy'),
        name='xgb_train').apply(mount_v3io())
    
    # deploy the model using nuclio functions
    srvfn = new_model_server('mysrv3', model_class='XGBoostModel', filename='nuclio_serving.ipynb')
    deploy = srvfn.with_v3io('User','~/').deploy_step(project = 'xgb', models={'netops_v1': train.outputs['model']}, dashboard='http://10.233.3.121:8070')
    
    # feed 1st step results into the secound step
    plot = fn.to_step(
        NewTask(handler='plot_iter', out_path=artifacts_path, 
                inputs={'iterations': train.outputs['iteration_results']}), 
        name='plot').apply(mount_v3io()) 
    

### Create a KubeFlow client and submit the pipeline with parameters

In [32]:
# for debug generate the pipeline dsl
#kfp.compiler.Compiler().compile(xgb_pipeline, 'mlrunpipe.yaml')

In [33]:
client = kfp.Client(namespace='default-tenant')
arguments = {'eta': [0.05, 0.10, 0.20, 0.30], 'gamma': [0.0, 0.1, 0.2, 0.3]}
run_result = client.create_run_from_pipeline_func(xgb_pipeline, arguments, run_name='xgb 1', experiment_name='xgb')

### See the run status and results in the run database

In [18]:
# connect to the run db 
db = get_run_db().connect()

In [19]:
# query the DB with filter on workflow ID (only show this workflow) 
db.list_runs('', labels=f'workflow={run_result.run_id}').show()

uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...9f7bef,0,Sep 21 20:58:02,completed,plot,workflow=fb1d5d08-9a82-4b9e-8970-cd60b85d6717kind=localowner=roothost=my-xgboost-training-pipeline-l4npq-4037507665,iterations,,,myfig.html
...76fd2a,16,Sep 21 20:57:40,completed,xgb_train,workflow=fb1d5d08-9a82-4b9e-8970-cd60b85d6717kind=localowner=roothost=my-xgboost-training-pipeline-l4npq-4238145964,,eta=0.3gamma=0.3,accuracy=1.0,model
...76fd2a,15,Sep 21 20:57:40,completed,xgb_train,workflow=fb1d5d08-9a82-4b9e-8970-cd60b85d6717kind=localowner=roothost=my-xgboost-training-pipeline-l4npq-4238145964,,eta=0.2gamma=0.3,accuracy=0.9666666666666667,model
...76fd2a,14,Sep 21 20:57:40,completed,xgb_train,workflow=fb1d5d08-9a82-4b9e-8970-cd60b85d6717kind=localowner=roothost=my-xgboost-training-pipeline-l4npq-4238145964,,eta=0.1gamma=0.3,accuracy=0.9333333333333333,model
...76fd2a,13,Sep 21 20:57:40,completed,xgb_train,workflow=fb1d5d08-9a82-4b9e-8970-cd60b85d6717kind=localowner=roothost=my-xgboost-training-pipeline-l4npq-4238145964,,eta=0.05gamma=0.3,accuracy=0.9666666666666667,model
...76fd2a,12,Sep 21 20:57:39,completed,xgb_train,workflow=fb1d5d08-9a82-4b9e-8970-cd60b85d6717kind=localowner=roothost=my-xgboost-training-pipeline-l4npq-4238145964,,eta=0.3gamma=0.2,accuracy=0.9333333333333333,model
...76fd2a,11,Sep 21 20:57:39,completed,xgb_train,workflow=fb1d5d08-9a82-4b9e-8970-cd60b85d6717kind=localowner=roothost=my-xgboost-training-pipeline-l4npq-4238145964,,eta=0.2gamma=0.2,accuracy=0.9333333333333333,model
...76fd2a,10,Sep 21 20:57:39,completed,xgb_train,workflow=fb1d5d08-9a82-4b9e-8970-cd60b85d6717kind=localowner=roothost=my-xgboost-training-pipeline-l4npq-4238145964,,eta=0.1gamma=0.2,accuracy=0.9666666666666667,model
...76fd2a,9,Sep 21 20:57:39,completed,xgb_train,workflow=fb1d5d08-9a82-4b9e-8970-cd60b85d6717kind=localowner=roothost=my-xgboost-training-pipeline-l4npq-4238145964,,eta=0.05gamma=0.2,accuracy=0.9333333333333333,model
...76fd2a,8,Sep 21 20:57:39,completed,xgb_train,workflow=fb1d5d08-9a82-4b9e-8970-cd60b85d6717kind=localowner=roothost=my-xgboost-training-pipeline-l4npq-4238145964,,eta=0.3gamma=0.1,accuracy=0.9666666666666667,model
